In [38]:
# Define project information

import sys

PROJECT_ID = "gen-lang-client-0303567819"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# if not running on colab, try to get the PROJECT_ID automatically
if "google.colab" not in sys.modules:
    import subprocess

    PROJECT_ID = subprocess.check_output(
        ["gcloud", "config", "get-value", "project"], text=True
    ).strip()

print(f"Your project ID is: {PROJECT_ID}")

Your project ID is: gen-lang-client-0303567819


Your active configuration is: [personal]


In [39]:
import sys

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [40]:
from rich import print as rich_print
from rich.markdown import Markdown as rich_Markdown
from IPython.display import Markdown, display
from vertexai.generative_models import (
    Content,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Image,
    Part,
)
from vertexai.language_models import TextEmbeddingModel

# Suppress deprecation warning for vision_models (deprecated until June 2026)
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning, message=".*vision_models.*")
    warnings.filterwarnings("ignore", category=UserWarning, message=".*deprecated.*")
    from vertexai.vision_models import MultiModalEmbeddingModel

In [41]:
# Multimodal models: Choose based on your performance/cost needs

multimodal_model_2_0_flash = GenerativeModel(
    "gemini-2.0-flash-001"
) # Gemini latest Gemini 2.0 Flash Model

multimodal_model_15 = GenerativeModel(
    "gemini-1.5-pro-001"
)  # works with text, code, images, video(with or without audio) and audio(mp3) with 1M input context - complex reasoning

# Multimodal models: Choose based on your performance/cost needs
multimodal_model_15_flash = GenerativeModel(
    "gemini-1.5-flash-001"
)  # works with text, code, images, video(with or without audio) and audio(mp3) with 1M input context - faster inference

# Load text embedding model from pre-trained source
text_embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-005")

# Load multimodal embedding model from pre-trained source
# Suppress deprecation warning when loading the model
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning, message=".*vision_models.*")
    warnings.filterwarnings("ignore", category=UserWarning, message=".*deprecated.*")
    multimodal_embedding_model = MultiModalEmbeddingModel.from_pretrained(
        "multimodalembedding@001"
    )  # works with image, image with caption(~32 words), video, video with caption(~32 words)

/home/luizeng/Documents/fanshawe_repo/test-image-text-gemini/.venv/lib/python3.13/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
/home/luizeng/Documents/fanshawe_repo/test-image-text-gemini/.venv/lib/python3.13/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
E0000 00:00:1762616269.378261   35855 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1762616269.657345   35855 alts_credentials.cc:93] ALTS creds ignor

In [42]:
# Load text embedding model from pre-trained source
text_embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-005")

/home/luizeng/Documents/fanshawe_repo/test-image-text-gemini/.venv/lib/python3.13/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
E0000 00:00:1762616269.891412   35855 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [43]:
# NOTE: multimodal_embedding_model is already correctly defined in Cell 3
# Do not overwrite it with a GenerativeModel - it must be a MultiModalEmbeddingModel
# If you need a different generative model, use a different variable name
# multimodal_embedding_model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

In [44]:
from multimodal_qa_with_rag_utils import (
    get_document_metadata,
    set_global_variable,
)

set_global_variable("text_embedding_model", text_embedding_model)
set_global_variable("multimodal_embedding_model", multimodal_embedding_model)

In [ ]:
# CÉLULA 75.4 - 🔄 RECARREGAR MÓDULO (Execute se o erro persistir)
# Esta célula recarrega o módulo multimodal_qa_with_rag_utils com as correções mais recentes

import importlib
import multimodal_qa_with_rag_utils
importlib.reload(multimodal_qa_with_rag_utils)

# Re-importar as funções atualizadas
from multimodal_qa_with_rag_utils import (
    get_image_embedding_from_multimodal_embedding_model,
    get_gemini_response,
    set_global_variable
)

print("✅ Módulo recarregado com sucesso!")
print("📋 Agora execute a CÉLULA 75.5 (Diagnóstico) e depois a CÉLULA 76")


In [ ]:
# CÉLULA 75.5 - 🔍 DIAGNÓSTICO: Verificar se os modelos estão corretos
# Execute esta célula ANTES da CÉLULA 76 para verificar se tudo está OK

print("=== DIAGNÓSTICO DOS MODELOS ===\n")

# Verificar multimodal_embedding_model
try:
    print(f"📊 Tipo do multimodal_embedding_model: {type(multimodal_embedding_model)}")
    print(f"📋 Tem método get_embeddings? {hasattr(multimodal_embedding_model, 'get_embeddings')}")
    
    if hasattr(multimodal_embedding_model, 'get_embeddings'):
        print("✅ multimodal_embedding_model está CORRETO!")
    else:
        print("❌ multimodal_embedding_model está INCORRETO!")
        print("   Execute a Cell 4 novamente para carregar o modelo correto.")
        print(f"   Tipo encontrado: {type(multimodal_embedding_model).__name__}")
        print(f"   Tipo esperado: MultiModalEmbeddingModel")
except NameError:
    print("❌ multimodal_embedding_model não está definido!")
    print("   Execute a Cell 4 primeiro.")

# Verificar se a variável global foi definida
try:
    from multimodal_qa_with_rag_utils import multimodal_embedding_model as global_model
    print(f"\n📊 Tipo do modelo global no módulo: {type(global_model)}")
    print(f"📋 Tem método get_embeddings? {hasattr(global_model, 'get_embeddings')}")
    if hasattr(global_model, 'get_embeddings'):
        print("✅ Modelo global está CORRETO!")
    else:
        print("❌ Modelo global está INCORRETO!")
        print("   Execute a Cell 7 novamente para definir a variável global.")
except (NameError, AttributeError) as e:
    print(f"\n⚠️  Não foi possível verificar o modelo global: {e}")
    print("   Execute a Cell 7 para definir a variável global.")

print("\n" + "="*60)
print("📋 PRÓXIMOS PASSOS:")
print("1. Se tudo estiver ✅, execute a CÉLULA 76")
print("2. Se houver ❌, execute as células indicadas primeiro")
print("="*60)


In [ ]:
# CÉLULA 78 (OPCIONAL) - 📄 EXTRAIR MAIS IMAGENS DO PDF PARA COMPARAÇÃO
# Esta célula extrai imagens do map.pdf para ter mais dados para comparação

print("=== EXTRAINDO IMAGENS DO PDF PARA AMPLIAR DATASET ===\n")

import fitz  # PyMuPDF
import os

def extrair_imagens_do_pdf(pdf_path, output_dir="images/", prefixo="map"):
    """
    Extrai imagens de um PDF e salva na pasta de imagens
    """
    print(f"🔍 Processando PDF: {pdf_path}")
    
    if not os.path.exists(pdf_path):
        print(f"❌ PDF não encontrado: {pdf_path}")
        return []
    
    # Criar diretório se não existir
    os.makedirs(output_dir, exist_ok=True)
    
    # Abrir PDF
    doc = fitz.open(pdf_path)
    imagens_extraidas = []
    
    print(f"📊 PDF tem {len(doc)} páginas")
    
    for page_num in range(len(doc)):
        page = doc[page_num]
        images = page.get_images()
        
        print(f"📄 Página {page_num + 1}: {len(images)} imagens encontradas")
        
        for img_index, img in enumerate(images):
            try:
                # Extrair imagem
                xref = img[0]
                pix = fitz.Pixmap(doc, xref)
                
                # Converter para RGB se necessário
                if pix.colorspace and pix.colorspace.n > 3:
                    pix = fitz.Pixmap(fitz.csRGB, pix)
                
                # Nome do arquivo
                img_filename = f"{prefixo}_page_{page_num + 1}_img_{img_index + 1}.png"
                img_path = os.path.join(output_dir, img_filename)
                
                # Salvar imagem
                pix.save(img_path)
                imagens_extraidas.append(img_path)
                
                print(f"  ✅ Extraída: {img_filename}")
                
                pix = None  # Liberar memória
                
            except Exception as e:
                print(f"  ❌ Erro ao extrair imagem {img_index}: {e}")
                continue
    
    doc.close()
    print(f"\n🎉 Total de {len(imagens_extraidas)} imagens extraídas!")
    return imagens_extraidas

# Verificar quantas imagens temos atualmente
# Lista todos os arquivos para debug
all_files = os.listdir("images/")
print(f"📁 Todos os arquivos na pasta 'images/':")
for f in sorted(all_files):
    file_path = os.path.join("images/", f)
    if os.path.isfile(file_path):
        size = os.path.getsize(file_path)
        print(f"  - {f} ({size} bytes)")

# Contar imagens com extensões suportadas (case-insensitive)
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp', '.gif', '.JPG', '.JPEG', '.PNG', '.BMP')
current_images = [f for f in all_files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp', '.gif'))]
print(f"\n📊 Imagens encontradas na pasta: {len(current_images)}")
for img in sorted(current_images):
    print(f"  ✅ {img}")

if len(current_images) <= 1:
    print("🔄 Extraindo imagens do PDF para ter mais dados...")
    
    # Extrair do map.pdf se existir
    if os.path.exists("map/map.pdf"):
        imagens_extraidas = extrair_imagens_do_pdf("map/map.pdf", "images/", "map")
        
        if imagens_extraidas:
            print(f"\n✅ {len(imagens_extraidas)} novas imagens adicionadas!")
            print("🚀 Agora execute a CÉLULA 76 novamente para processar todas as imagens")
            print("   Depois execute a CÉLULA 70 para testar similaridade com mais dados")
        else:
            print("❌ Nenhuma imagem foi extraída do PDF")
    else:
        print("❌ Arquivo map/map.pdf não encontrado")
        
        # Verificar outros PDFs disponíveis
        print("\n🔍 Procurando outros PDFs...")
        pdf_paths = []
        for root, dirs, files in os.walk("."):
            for file in files:
                if file.lower().endswith('.pdf'):
                    pdf_paths.append(os.path.join(root, file))
        
        if pdf_paths:
            print("📋 PDFs encontrados:")
            for i, pdf_path in enumerate(pdf_paths[:3], 1):  # Mostrar apenas os 3 primeiros
                print(f"  {i}. {pdf_path}")
                
            # Processar o primeiro PDF encontrado
            if pdf_paths:
                primeiro_pdf = pdf_paths[0]
                print(f"\n🔄 Processando: {primeiro_pdf}")
                imagens_extraidas = extrair_imagens_do_pdf(primeiro_pdf, "images/", "doc")
                
                if imagens_extraidas:
                    print(f"\n✅ {len(imagens_extraidas)} imagens extraídas de {primeiro_pdf}!")
                    print("🚀 Execute a CÉLULA 76 novamente para processar todas as imagens")
        else:
            print("❌ Nenhum PDF encontrado para extrair imagens")
            
else:
    print("✅ Já há múltiplas imagens na pasta")
    print("Execute a CÉLULA 76 para processar todas e depois a CÉLULA 70 para testar similaridade")

# Mostrar status final - usar a mesma lógica de contagem
all_files_final = os.listdir("images/")
final_images = [f for f in all_files_final if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp', '.gif'))]
print(f"\n📊 STATUS FINAL: {len(final_images)} imagens na pasta 'images/'")
print("📋 Imagens encontradas:")
for img in sorted(final_images):
    print(f"  - {img}")

if len(final_images) > 1:
    print("🎉 Pronto para testar busca por similaridade!")
    print("📋 PRÓXIMOS PASSOS:")
    print("  1. Execute CÉLULA 76 (processar todas as imagens)")
    print("  2. Execute CÉLULA 70 (busca por similaridade)")
    print("  3. Execute CÉLULA 71 (análise contextual)")
else:
    print("⚠️  Ainda há apenas 1 imagem. Adicione mais imagens manualmente na pasta 'images/'")


=== EXTRAINDO IMAGENS DO PDF PARA AMPLIAR DATASET ===

📁 Buscando imagens na pasta 'images/' e subdiretórios...

📂 Arquivos na pasta 'images/' (diretório raiz):
  - A1.png (73671 bytes) [extensão: .png]
  - B2_room.jpeg (103810 bytes) [extensão: .jpeg]

📊 Total de imagens encontradas: 2
✅ Imagens detectadas:
  - A1.png
  - B2_room.jpeg
✅ Já há múltiplas imagens na pasta
Execute a CÉLULA 76 para processar todas e depois a CÉLULA 70 para testar similaridade

📊 STATUS FINAL:
📊 Total: 2 imagens na pasta 'images/'
📋 Imagens encontradas:
  - A1.png
  - B2_room.jpeg
🎉 Pronto para testar busca por similaridade!
📋 PRÓXIMOS PASSOS:
  1. Execute CÉLULA 76 (processar todas as imagens)
  2. Execute CÉLULA 70 (busca por similaridade)
  3. Execute CÉLULA 71 (análise contextual)


In [55]:
# CÉLULA 75 (NOVO) - 📂 PROCESSAMENTO DIRETO DE IMAGENS DA PASTA
# Função para ler todas as imagens da pasta images/ e gerar embeddings para RAG
import os
import glob
import pandas as pd
import numpy as np
from pathlib import Path
from multimodal_qa_with_rag_utils import (
    get_image_embedding_from_multimodal_embedding_model,
    get_gemini_response
)
def processar_imagens_da_pasta(
    pasta_imagens="images/",
    embedding_size=512,
    gerar_descricoes=True,
    formatos_suportados=['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']
):
    """
    Processa todas as imagens de uma pasta, gerando embeddings e descrições para RAG
    
    Args:
        pasta_imagens: Caminho da pasta com imagens
        embedding_size: Tamanho do embedding (128, 256, 512, 1408)
        gerar_descricoes: Se deve gerar descrições das imagens com Gemini
        formatos_suportados: Lista de formatos de imagem aceitos
    
    Returns:
        pd.DataFrame: DataFrame compatível com o sistema RAG existente
    """
    print(f"🔍 PROCESSANDO IMAGENS DA PASTA: {pasta_imagens}")
    print("="*60)
    
    # Verificar se a pasta existe
    if not os.path.exists(pasta_imagens):
        print(f"❌ Pasta '{pasta_imagens}' não encontrada!")
        return pd.DataFrame()
    
    # Encontrar todas as imagens na pasta
    imagens_encontradas = []
    for formato in formatos_suportados:
        pattern = os.path.join(pasta_imagens, f"*{formato}")
        imagens_encontradas.extend(glob.glob(pattern))
        pattern = os.path.join(pasta_imagens, f"*{formato.upper()}")
        imagens_encontradas.extend(glob.glob(pattern))
    
    # Remover duplicatas
    imagens_encontradas = list(set(imagens_encontradas))
    
    if not imagens_encontradas:
        print(f"❌ Nenhuma imagem encontrada na pasta '{pasta_imagens}'")
        print(f"Formatos suportados: {formatos_suportados}")
        return pd.DataFrame()
    
    print(f"📊 Encontradas {len(imagens_encontradas)} imagens:")
    for img in imagens_encontradas:
        print(f"  - {os.path.basename(img)}")
    
    # Lista para armazenar dados processados
    dados_imagens = []
    
    # Prompt para descrição das imagens
    prompt_descricao = """Analise esta imagem detalhadamente e forneça uma descrição precisa.
    Inclua:
    - O que você vê na imagem
    - Elementos principais e detalhes importantes
    - Texto visível (se houver)
    - Tipo de imagem (mapa, diagrama, foto, etc.)
    - Informações relevantes para busca e recuperação
    
    Seja específico e detalhado para facilitar buscas futuras."""
    
    print(f"\n🚀 PROCESSANDO CADA IMAGEM...")
    print("="*60)
    
    for i, caminho_imagem in enumerate(imagens_encontradas, 1):
        nome_arquivo = os.path.basename(caminho_imagem)
        print(f"\n📸 PROCESSANDO {i}/{len(imagens_encontradas)}: {nome_arquivo}")
        
        try:
            # 1. Gerar embedding da imagem
            print("  🔄 Gerando embedding...")
            image_embedding = get_image_embedding_from_multimodal_embedding_model(
                image_uri=caminho_imagem,
                embedding_size=embedding_size,
             model=multimodal_embedding_model,
                return_array=True
            )
            print(f"  ✅ Embedding gerado: shape {image_embedding.shape}")
            
            # 2. Gerar descrição da imagem (se solicitado)
            descricao = ""
            if gerar_descricoes:
                print("  🤖 Gerando descrição com Gemini...")
                try:
                    from vertexai.generative_models import Image as GeminiImage
                    imagem_gemini = GeminiImage.load_from_file(caminho_imagem)
                    
                    descricao = get_gemini_response(
                        multimodal_model_2_0_flash,
                        model_input=[prompt_descricao, imagem_gemini],
                        stream=False,
                    )
                    print(f"  ✅ Descrição gerada: {len(descricao)} caracteres")
                    
                except Exception as desc_error:
                    print(f"  ⚠️  Erro ao gerar descrição: {desc_error}")
                    descricao = f"Imagem: {nome_arquivo}"
            
            # 3. Gerar embedding da descrição (para compatibilidade com RAG)
            text_embedding = None
            if descricao:
                try:
                    from multimodal_qa_with_rag_utils import get_text_embedding_from_text_embedding_model
                    text_embedding = get_text_embedding_from_text_embedding_model(descricao)
                    print("  ✅ Text embedding da descrição gerado")
                except Exception as text_emb_error:
                    print(f"  ⚠️  Erro ao gerar text embedding: {text_emb_error}")
            
            # 4. Criar registro compatível com o sistema existente
            registro = {
                'file_name': f"pasta_images_{nome_arquivo}",  # Nome único
                'page_num': 1,  # Imagens individuais = página 1
                'img_num': i,
                'img_path': caminho_imagem,
                'img_desc': descricao,
                'mm_embedding_from_img_only': image_embedding.tolist(),  # Compatibilidade
                'text_embedding_from_image_description': text_embedding if text_embedding else None,
                'source_type': 'pasta_imagens',  # Identificar origem
                'original_filename': nome_arquivo
            }
            
            dados_imagens.append(registro)
            print(f"  ✅ Processamento concluído para {nome_arquivo}")
            
        except Exception as e:
            print(f"  ❌ Erro ao processar {nome_arquivo}: {e}")
            continue
    
    # Criar DataFrame
    if dados_imagens:
        df_imagens = pd.DataFrame(dados_imagens)
        print(f"\n🎉 PROCESSAMENTO CONCLUÍDO!")
        print(f"📊 DataFrame criado com {len(df_imagens)} imagens processadas")
        print(f"📋 Colunas: {list(df_imagens.columns)}")
        
        return df_imagens
    else:
        print(f"\n❌ Nenhuma imagem foi processada com sucesso")
        return pd.DataFrame()
print("✅ Função 'processar_imagens_da_pasta' criada com sucesso!")



✅ Função 'processar_imagens_da_pasta' criada com sucesso!


In [56]:
# CÉLULA 76 (EXECUTAR) - 🚀 PROCESSAMENTO DAS IMAGENS DA PASTA images/
# Executa o processamento de todas as imagens e cria o image_metadata_df

print("=== PROCESSAMENTO COMPLETO DA PASTA IMAGES/ ===\n")

# Executar o processamento das imagens
try:
    image_metadata_df = processar_imagens_da_pasta(
        pasta_imagens="images/",
        embedding_size=512,
        gerar_descricoes=True,  # Gerar descrições detalhadas com Gemini
        formatos_suportados=['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']
    )
    
    if not image_metadata_df.empty:
        print(f"\n🎉 SUCESSO TOTAL!")
        print(f"📊 image_metadata_df criado com {len(image_metadata_df)} imagens")
        
        # Mostrar resumo das imagens processadas
        print(f"\n📋 RESUMO DAS IMAGENS PROCESSADAS:")
        print("="*50)
        for idx, row in image_metadata_df.iterrows():
            print(f"\n🖼️  Imagem {idx + 1}:")
            print(f"  📁 Arquivo: {row['original_filename']}")
            print(f"  📂 Caminho: {row['img_path']}")
            print(f"  📊 Embedding shape: {len(row['mm_embedding_from_img_only'])}")
            
            # Mostrar início da descrição
            desc = row['img_desc']
            if desc and len(desc) > 10:
                print(f"  📝 Descrição: {desc[:150]}{'...' if len(desc) > 150 else ''}")
        
        # Verificar compatibilidade com sistema RAG existente
        print(f"\n✅ COMPATIBILIDADE COM SISTEMA RAG:")
        colunas_necessarias = ['img_path', 'mm_embedding_from_img_only', 'img_desc', 'file_name', 'page_num']
        for col in colunas_necessarias:
            if col in image_metadata_df.columns:
                print(f"  ✅ {col}: OK")
            else:
                print(f"  ❌ {col}: FALTANDO")
        
        # Salvar para uso futuro (opcional)
        try:
            image_metadata_df.to_pickle("image_metadata_from_folder.pkl")
            print(f"\n💾 DataFrame salvo em 'image_metadata_from_folder.pkl'")
        except Exception as save_error:
            print(f"\n⚠️  Não foi possível salvar: {save_error}")
        
        print(f"\n🚀 PRÓXIMOS PASSOS:")
        print(f"1. Agora você pode executar a CÉLULA 70 (Validação)")
        print(f"2. Depois executar a CÉLULA 71 (Análise Contextual)")
        print(f"3. O sistema RAG está pronto para perguntas sobre as imagens!")
        
    else:
        print(f"\n❌ FALHA: Nenhuma imagem foi processada")
        print(f"Verifique se:")
        print(f"- A pasta 'images/' existe")
        print(f"- Há imagens válidas na pasta")
        print(f"- Os modelos estão carregados corretamente")

except Exception as e:
    print(f"❌ ERRO NO PROCESSAMENTO: {e}")
    import traceback
    traceback.print_exc()
    
    print(f"\n💡 POSSÍVEIS SOLUÇÕES:")
    print(f"- Verifique se os modelos estão carregados")
    print(f"- Verifique se a pasta 'images/' existe")
    print(f"- Execute as células de setup dos modelos primeiro")


=== PROCESSAMENTO COMPLETO DA PASTA IMAGES/ ===

🔍 PROCESSANDO IMAGENS DA PASTA: images/
📊 Encontradas 2 imagens:
  - B2_room.jpeg
  - A1.png

🚀 PROCESSANDO CADA IMAGEM...

📸 PROCESSANDO 1/2: B2_room.jpeg
  🔄 Gerando embedding...
  ✅ Embedding gerado: shape (512,)
  🤖 Gerando descrição com Gemini...
  ⚠️  Erro ao gerar descrição: 'GenerationResponse' object is not iterable
  ✅ Text embedding da descrição gerado
  ✅ Processamento concluído para B2_room.jpeg

📸 PROCESSANDO 2/2: A1.png
  🔄 Gerando embedding...
  ✅ Embedding gerado: shape (512,)
  🤖 Gerando descrição com Gemini...
  ⚠️  Erro ao gerar descrição: 'GenerationResponse' object is not iterable
  ✅ Text embedding da descrição gerado
  ✅ Processamento concluído para A1.png

🎉 PROCESSAMENTO CONCLUÍDO!
📊 DataFrame criado com 2 imagens processadas
📋 Colunas: ['file_name', 'page_num', 'img_num', 'img_path', 'img_desc', 'mm_embedding_from_img_only', 'text_embedding_from_image_description', 'source_type', 'original_filename']

🎉 SUCESSO

In [67]:

# Este código permite que você faça perguntas sobre as imagens usando o embedding e o Gemini

import numpy as np
from multimodal_qa_with_rag_utils import get_text_embedding_from_text_embedding_model

def responder_pergunta_sobre_imagem(pergunta, image_metadata_df, text_embedding_model, gemini_model, top_k=3):
    """
    Dado uma pergunta, retorna a resposta do Gemini baseada nas imagens mais relevantes.
    
    Args:
        pergunta: Texto da pergunta
        image_metadata_df: DataFrame com metadados das imagens (deve ter coluna 'text_embedding_from_image_description')
        text_embedding_model: Modelo de embedding de texto (TextEmbeddingModel)
        gemini_model: Modelo Gemini para gerar respostas
        top_k: Número de imagens mais relevantes para usar no contexto
    """

    # Gere o embedding da pergunta usando o modelo de texto
    pergunta_embedding = get_text_embedding_from_text_embedding_model(pergunta, return_array=True)
    pergunta_embedding = np.array(pergunta_embedding)

    # Calcule similaridades usando os embeddings de texto das descrições das imagens
    # Primeiro, precisamos converter os embeddings de texto das descrições para arrays
    text_embeddings_list = []
    valid_indices = []
    
    for idx, row in image_metadata_df.iterrows():
        text_emb = row.get('text_embedding_from_image_description')
        if text_emb is not None:
            if isinstance(text_emb, list):
                text_embeddings_list.append(np.array(text_emb))
                valid_indices.append(idx)
            elif isinstance(text_emb, np.ndarray):
                text_embeddings_list.append(text_emb)
                valid_indices.append(idx)
    
    if not text_embeddings_list:
        return "❌ Erro: Nenhum embedding de texto encontrado nas descrições das imagens. Execute a CÉLULA 76 primeiro para processar as imagens."
    
    # Calcular similaridades
    text_embeddings = np.stack(text_embeddings_list)
    similarities = np.dot(text_embeddings, pergunta_embedding) / (
        np.linalg.norm(text_embeddings, axis=1) * np.linalg.norm(pergunta_embedding) + 1e-8
    )
    top_indices_local = np.argsort(similarities)[-top_k:][::-1]  # índices locais
    top_indices = [valid_indices[i] for i in top_indices_local]  # índices originais do DataFrame

    # Monte o contexto para o Gemini
    contexto_imgs = []
    imagens_para_gemini = []
    for idx in top_indices:
        row = image_metadata_df.iloc[idx]
        contexto_imgs.append(
            f"Arquivo: {row['file_name']} (página {row['page_num']}), descrição: {row['img_desc']}"
        )
        # Carregar a imagem para incluir no contexto visual
        try:
            from vertexai.generative_models import Image as GeminiImage
            img_obj = GeminiImage.load_from_file(row['img_path'])
            imagens_para_gemini.append(img_obj)
        except Exception as e:
            print(f"⚠️  Não foi possível carregar imagem {row['img_path']}: {e}")
    
    contexto = "\n".join(contexto_imgs)
    prompt = (
        "Considere as seguintes descrições de imagens extraídas e responda a pergunta:"
        f"\n\n{contexto}\n\nPergunta: {pergunta}\nResposta:"
    )

    # Chame o Gemini com texto e imagens
    try:
        if imagens_para_gemini:
            # Incluir imagens no contexto
            model_input = [prompt] + imagens_para_gemini
        else:
            model_input = [prompt]
        
        resposta = gemini_model.generate_content(model_input)
        # Pode ser "resposta.text" ou apenas "resposta" dependendo do SDK
        if hasattr(resposta, 'text'):
            return resposta.text
        else:
            return str(resposta)
    except Exception as e:
        return f"❌ Erro ao gerar resposta: {e}"

# Exemplo de uso (corrigido para usar text_embedding_model):
resposta = responder_pergunta_sobre_imagem("Qual mapa mostra a América do Sul?", image_metadata_df, text_embedding_model, multimodal_model_2_0_flash)
print("Resposta:", resposta)




Resposta: Nenhuma das imagens representa um mapa da América do Sul. As imagens mostram plantas baixas de edifícios.
